# _Spectrogram_, _Cepstrogram_ y _LPCtrogram_

Procesamiento del Habla (66.46) - FIUBA

### Librerías usadas

In [14]:
import numpy as np

from matplotlib import pyplot as plt

from scipy.io.wavfile import read as wavread, write as wavwrite
from scipy.signal import spectrogram, freqz, lfilter
from scipy.linalg import toeplitz, inv

### Funciones auxiliares

In [17]:
# loads a WAV file
def load(path):
    
    Fs, x = wavread(path)

    if x.dtype == 'int16':
        nb_bits = np.float(16)
        x = x / 2**(nb_bits-1)
    elif x.dtype == 'int32':
        nb_bits = np.float(32)
        x = x / 2**(nb_bits-1)
    elif x.dtype == 'uint8':
        x = x / 2**8

    x = x.astype(np.float64)

    return Fs, x

# computes the FFT
def fft(x, nfft=None):
    return np.fft.fft(x, len(x) if nfft is None else nfft)

# computes the IFFT
def ifft(x):
    return np.fft.ifft(x)

# plot x-trogram
def plot_xtrogram(t, f, Sxx, title, name):

    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)

    fig.suptitle(title)
    ax.pcolormesh(t, f, np.log(np.abs(Sxx)))
    ax.set(xlabel='Time [s]', ylabel='Frequecy [Hz]')
    ax.plot()
    
    plt.show()
    plt.savefig("{}.png".format(name))

### _Plotting_

In [18]:
def plot_spectrogram(x, Fs):
    
    n_perseg = int(0.025 * Fs)
    window = 'hanning'
    detrend='constant'
    return_onesided=True,
    scaling='density'
    axis=-1
    mode='psd'
    nfft = n_perseg * 8

    f, t, Sxx = spectrogram(x, window=window, fs=Fs,
                            nperseg=n_perseg, noverlap=n_overlap, nfft=Nfft,
                            detrend=detrend, return_onesided=return_onesided,
                            scaling=scaling, axis=axis, mode=mode)
    
    plot_xtrogram(t, f, Sxx)

In [ ]:
def plot_cepstrogram(x, Fs):
    
    H_mat = cepstrogram(x, Fs)
        
    NF, NT = H_mat.shape

    f = np.arange(NF) * Fs / NF
    t = np.arange(NT) * 0.010
    
    plot_xtrogram(t, f, H_mat)

In [12]:
fs, x = load("./waves/fantasia.wav")